In [14]:
import pandas as pd
import numpy as np
import random as rd
import os

In [15]:
# Setup the dimension table

data1 = {"key":["CA", "NY", "WA", "ON", "QU"],
        "name":["California", "new York", "Washington", "Ontario", "Quebec"],
        "country":["USA", "USA", "USA", "Canada", "Canada"]}
state_table = pd.DataFrame(data1)

In [16]:
data2 = {"key":range(1,13),
         "desc":["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
         "quarter":["Q1","Q1","Q1","Q2","Q2","Q2","Q3","Q3","Q3","Q4","Q4","Q4"]}
month_table = pd.DataFrame(data2)

In [17]:
data3 = {"key":["Printer", "Tablet", "Laptop"],
         "price":[225, 570, 1120]}
prod_table = pd.DataFrame(data3)

In [18]:
# Function to generate the Sales table
def gen_sales(no_of_recs):
    # Generate transaction data randomly
    loc = rd.choices(state_table.key,weights = [2, 2, 1, 1, 1],k=no_of_recs)
    time_month = rd.choices(month_table.key,k=no_of_recs)
    time_year = rd.choices([2018,2019],k=no_of_recs)
    prod = rd.choices(prod_table.key,k= no_of_recs,weights = [1, 3, 2])
    unit = rd.choices([1,2],k=no_of_recs,weights = [10, 3])
    
    sales = pd.DataFrame({'month':time_month,
                      'year':time_year,
                      'location':loc,
                      'product':prod,
                      'unit':unit
                         }
                      )
    
      # Sort the records by time order
    sales = sales.sort_values(by = ['year','month'])
    return(sales)

In [19]:
sales_fact = gen_sales(500)
sales_fact["value"] = 1
sales_fact['quarter'] = 1
sales_fact['country'] = 1

In [20]:
def amount(cols):
    value = cols[0]
    product = cols[1]
    while value == 1:
        if  product == "Tablet":
            return 570
        elif product == "Printer":
            return 225
        else:
            return 1120
        
def quarter(cols):
    quarter = cols[0]
    month = cols[1]
    while quarter == 1:
        if  month <= 3:
            return 'Q1'
        elif month <= 6:
            return 'Q2'
        elif month <= 9:
            return 'Q3'
        else:
            return 'Q4' 

def country(cols):
    country = cols[0]
    location = cols[1]
    while country == 1:
        if  location == "CA":
            return 'USA'
        elif location == "NY":
            return 'USA'
        elif location == "WA":
            return 'USA'
        elif location == "ON":
            return 'Canada'
        else:
            return 'Canada'        

In [21]:
sales_fact["quarter"] = sales_fact[['quarter','month']].apply(quarter,axis=1)
sales_fact["country"] = sales_fact[['country','location']].apply(country,axis=1)

In [22]:
sales_fact["value"] = sales_fact[['value','product']].apply(amount,axis=1)
sales_fact['amount'] = sales_fact["value"]*sales_fact['unit']

In [24]:
sales_fact.to_csv('sales.csv')
sale = os.path.abspath("sales.csv")
revenue =  pd.read_csv(sale)

In [25]:
revenue.drop(['Unnamed: 0','unit','value'],axis=1,inplace=True)

In [26]:
revenue = revenue[['month', 'quarter','year','location','country','product','amount']]

In [27]:
revenue.head(20)

,month,quarter,year,location,country,product,amount
0,1,Q1,2018,CA,USA,Tablet,570
1,1,Q1,2018,NY,USA,Tablet,570
2,1,Q1,2018,CA,USA,Laptop,1120
3,1,Q1,2018,WA,USA,Laptop,1120
4,1,Q1,2018,NY,USA,Tablet,1140
5,1,Q1,2018,ON,Canada,Printer,225
6,1,Q1,2018,NY,USA,Tablet,570
7,1,Q1,2018,QU,Canada,Tablet,570
8,1,Q1,2018,CA,USA,Laptop,2240
9,1,Q1,2018,CA,USA,Tablet,570


# "Dice" is about limited each dimension to a certain range of values, while keeping the number of dimensions the same in the resulting cube.  For example, we can focus in sales happening in [Jan/ Feb/Mar, Laptop/Tablet, CA/NY].

In [28]:
dc = revenue[(revenue['year'] ==2018) & (revenue['location'] == 'CA') & 
             ((revenue['product'] =='Laptop') | (revenue['product']=='Tablet')) &
            ((revenue['month']==1) | (revenue['month']==2) | (revenue['month']==3))]
dc.groupby(['year','product','month']).sum()

amount
year product month        
2018 Laptop  1        6720
             2        1120
             3        4480
     Tablet  1        2280
             2        1140
             3        1710

# "Rollup" is about applying an aggregation function to collapse a number of dimensions.  For example, we want to focus in the annual revenue for each product and collapse the location dimension (ie: we don't care where we sold our product). 

In [29]:
revenue.groupby(['year']).sum().drop(['month'],axis=1)

,amount
year,
2018,210090
2019,210490


# "Slice" is about fixing certain dimensions to analyze the remaining dimensions.  For example, we can focus in the sales happening in "2019", "Feb", or we can focus in the sales happening in "2019", "Jan", "Tablet".

In [30]:
revenue[(revenue['year'] ==2019) & (revenue['month'] == 2)].head()

,month,quarter,year,location,country,product,amount
263,2,Q1,2019,CA,USA,Laptop,2240
264,2,Q1,2019,QU,Canada,Tablet,570
265,2,Q1,2019,WA,USA,Laptop,2240
266,2,Q1,2019,NY,USA,Tablet,570
267,2,Q1,2019,CA,USA,Tablet,570


# "Drilldown" is the reverse of "rollup" and applying an aggregation function to a finer level of granularity.  For example, we want to focus in the annual and monthly revenue for each product and collapse the location dimension (ie: we don't care where we sold our product).

In [33]:
revenues = revenue[revenue['product'] == 'Laptop']
revenues.groupby(['year']).sum().drop(['month'],axis=1)

,amount
year,
2018,114240
2019,101920


# "Pivot" is about analyzing the combination of a pair of selected dimensions.  For example, we want to analyze the revenue by year and month.  Or we want to analyze the revenue by product and location.

In [34]:
revenue.pivot_table(index='product',columns='location',values='amount')

location,CA,NY,ON,QU,WA
product,,,,,
Laptop,1319.111111,1292.307692,1440.000000,1368.888889,1456.000000
Printer,290.625000,289.285714,255.000000,225.000000,246.428571
Tablet,686.746988,762.794118,730.769231,712.500000,712.500000
